This is like a utils module for my project, including useful functions, mainly for data preparation

In [3]:
#  Mount google drive
if __name__ == '__main__' and '__file__' not in globals():
  from google.colab import drive
  drive.mount('/content/gdrive/')
  %cd gdrive/'My Drive'/Internship/Project

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/Internship/Project'
/content/gdrive/My Drive/Internship/Project


In [4]:
import numpy as np
import h5py
import os
from scipy.io import loadmat
from sklearn.preprocessing import RobustScaler
from pickle import dump, load
from tqdm.notebook import trange, tqdm
from tensorflow.keras.models import Model, load_model, model_from_json
import tensorflow as tf
import matplotlib.pyplot as plt
import numba
from scipy import fftpack
# import scipy.fftpack._fftpack as sff

In [5]:
%load_ext Cython

In [6]:
%%cython
cimport numpy as cnp
import numpy as np

'''
These functions are based on the impressions that there were many leading and trailing zeros in the recordings because of
recording errors. In the end, they were useful to remove the beginning of the recordings for which the quality labels were all zero 
because they haven't started recording with the invasive technique yet.
'''


# It counts how many zeros exist in an array before the first one
cpdef int c_count_leading_zeros(cnp.ndarray a, thr=1e-6):
    cdef int elements = a.size
    cdef int i = 0
    cdef int count = 0
    while i < elements:
        if np.abs(a[i]) < thr:
            count += 1
        else:
            return count
        i += 1
    return count

# it counts how many consecutive zeros exist in the end of an array
cpdef int c_count_trailing_zeros(cnp.ndarray a):
    return c_count_leading_zeros(np.flip(a))

# It removes leading and trailing zeros from an array
cpdef cnp.ndarray c_trim_zeros_NEMO(cnp.ndarray arr):
    # minimum number of leading zeros
    max_lead = -np.inf
    max_trail = -np.inf

    cdef int i
    for i in range(arr.shape[0]):
        lead = c_count_leading_zeros(arr[i])
        trail = c_count_trailing_zeros(arr[i])

        if lead > max_lead:
            max_lead = lead
        if trail > max_trail:
            max_trail = trail
            
    if max_lead == -np.inf:
        start = 0
    else:
        start = max_lead
        
    if max_trail == -np.inf:
        end = arr.shape[1]
    else:
        end = arr.shape[1] - max_trail

    return arr[:,start:end]


In [7]:
# Python wrapper of the previous cython functions
def count_leading_zeros(a):
  return(c_count_leading_zeros(a))

def count_trailing_zeros(a):
  return c_count_trailing_zeros(a)

def trim_zeros_NEMO(a):
  return c_trim_zeros_NEMO(a)

In [8]:
%%cython
cimport numpy as cnp
import numpy as np
import h5py
import os
from tqdm.notebook import trange, tqdm
from __main__ import trim_zeros_NEMO

'''
Create dataset from the NEMO dataset
path: path to the train and test directories
snippet_len: length of a snippet (number of timesteps in an LSTM network)
RETURN: array in appropriate format to be fed to the NN
''' 
cpdef cnp.ndarray create_dataset(path = r'Data/fetal_quality_assessment/train', snippet_len=125):

  cdef cnp.ndarray signal
  cdef int leftovers
  cdef int i
  cdef int iters
  cdef cnp.ndarray dataset = np.zeros(shape=(0,snippet_len, 4))
  cdef cnp.ndarray temp_dataset

  filenames = os.listdir(path)

  for name in tqdm(filenames):

    file_signal = h5py.File(path+'/'+name, 'r')
    signal = file_signal['data'][()]
    file_signal.close()

    # trim leading and trailing zeros
    signal = trim_zeros_NEMO(signal)

    # make it an integer multiple of snippet_len
    leftovers = signal.shape[1]%snippet_len
    if leftovers != 0:
      signal = signal[:,:-leftovers]

    temp_dataset = np.zeros((signal.shape[1]//snippet_len-1, snippet_len, signal.shape[0]))

    iters = signal.shape[1]//snippet_len
    for i in range(0, iters):
      temp_dataset[i] = signal[:,i*snippet_len:(i+1)*snippet_len].T
    
    dataset = np.vstack((dataset, temp_dataset))

  return dataset

In [9]:
%%cython
cimport numpy as cnp
import numpy as np
import h5py
import os
from tqdm.notebook import trange, tqdm
from __main__ import trim_zeros_NEMO

'''
Create dataset from the a single NEMO file
name: filename including path
snippet_len: length of a snippet (number of timesteps in an LSTM network)
RETURN: array in appropriate format to be fed to the NN
''' 
cpdef create_dataset_from_file(name, snippet_len=125, sliding_window=False, trim_zeros=True, return_leftovers=False):
# def create_dataset_from_file(name, snippet_len=125, sliding_window=False, trim_zeros=True, return_leftovers=False):

  cdef cnp.ndarray signal
  cdef int leftovers
  cdef int i
  cdef int iters
  cdef cnp.ndarray dataset = np.zeros(shape=(0,snippet_len, 4))
  cdef cnp.ndarray temp_dataset

  file_signal = h5py.File(name, 'r')
  signal = file_signal['data'][()]
  file_signal.close()

  # trim leading and trailing zeros
  if trim_zeros:
    signal = trim_zeros_NEMO(signal)

  # make it an integer multiple of snippet_len
  leftovers = signal.shape[1]%snippet_len
  if leftovers != 0:
    signal = signal[:,:-leftovers]

  if not sliding_window or snippet_len==125:

    dataset = np.zeros((signal.shape[1]//snippet_len, snippet_len, signal.shape[0]))

    iters = signal.shape[1]//snippet_len-1
    for i in range(0, iters):
      dataset[i] = signal[:,i*snippet_len:(i+1)*snippet_len].T
    
  
  else:

    dataset = np.zeros(((signal.shape[1]-snippet_len)//125, snippet_len, signal.shape[0]))

    i = 0
    while i*125+snippet_len < signal.shape[1]:
      s = signal[:,i*125:i*125+snippet_len].T
      dataset[i,:,:] = s
      i += 1

  if return_leftovers:    
    return dataset, leftovers
  else:
    return dataset


In [10]:
# remove the nan values from the quality arrays, replace them with low quality
def remove_nan(quality):
  quality[np.where(np.isnan(quality) == True)] = 0
  return quality

In [11]:
# remove the inf values from the recording arrays, replace them with the non infinite maximum/minimum
def remove_inf(signal):

  no_inf = np.copy(signal)
  no_inf[np.where(np.isinf(signal) == True)] = 0
  maximum = np.max(no_inf)
  minimum = np.min(no_inf)

  signal[np.where(signal == np.inf)] = maximum
  signal[np.where(signal == -np.inf)] = minimum

  return signal

In [12]:
# It creates a Robust scaler from the data
def create_scaler(data):
  scaler = RobustScaler()
  scaler.fit(data.reshape(-1,4))
  return scaler

In [13]:
# It scales the data using the given scaler in the format appropriate for training our networks
def scale_data(data, scaler, snippet_len=125):
  return scaler.transform(data.reshape(-1,4)).reshape(-1, snippet_len, 4)

In [14]:
%%cython
cimport numpy as cnp
import numpy as np
import h5py
from tqdm.notebook import trange, tqdm

'''
(Not used in the end because of the spread of low quality recordings)
This function will turn the dataset in a clean version where the recordings do not contain many low quality data points.
Filenames include paths. 
max_bad_points: the maximum allowed number of "bad" datapoints in a recording
min_duration: the minimum duration of a recording in seconds,
data_filename: path+name of the data file 
quality_filename: path+name of the data file 
snippet_len: how many datapoints correspond to one quality point
RETURN: a list of recordings (arrays)
'''
cpdef list create_clean_train_set(int max_bad_points, int min_duration, str data_filename, str quality_filename, int snippet_len=125):

  cdef int bad_counter = 0
  cdef int starting_index = 0
  cdef int ending_index = 0
  cdef int i
  cdef int total_counter = 0
  cdef list signals = []

  file_signal = h5py.File(quality_filename, 'r')
  cdef cnp.ndarray quality = file_signal['data'][()][0]
  file_signal.close()

  quality[np.where(np.isnan(quality) == True)] = 0

  file_signal = h5py.File(data_filename, 'r')
  cdef cnp.ndarray data = file_signal['data'][()]
  file_signal.close()

  for i in range(len(quality)):
    
    total_counter += 1

    if quality[i] == 0:
      bad_counter += 1

    if bad_counter <= max_bad_points and total_counter/4 >= min_duration:

      ending_index = i
      signals.append(data[:, starting_index*snippet_len : (ending_index+1)*snippet_len])

      starting_index = i+1
      total_counter = 0
      bad_counter = 0
        
  return signals


In [15]:
# Not used in the end
def clean_data(max_bad_points, min_duration, path = r'Data/fetal_quality_assessment'):
  filenames_with_ending = os.listdir(path+'/train')
  filenames = []
  for n in filenames_with_ending:
    filenames.append(n[:-12])

  for name in tqdm(filenames):
    signals = create_clean_train_set(max_bad_points, min_duration, path+'/train/'+name+'.fetalSignal', path+'/quality_files/'+name+'.quality')

    for i in range(len(signals)):

      recording = signals[i]

      hf = h5py.File(r'Data/clean_data/{}.h5'.format(name+'__'+str(i)), 'w')
      hf.create_dataset('data', data=recording)
      hf.close()

In [16]:
# clean_data(max_bad_points=50, min_duration=30, path=r'Data/fetal_quality_assessment')

In [17]:
# serialize Keras model to JSON and save the weights in one signle function
def save_model(model, name):
  model_json = model.to_json()
  with open(name+".json", "w") as json_file:
      json_file.write(model_json)
  # serialize weights to HDF5
  model.save_weights(name+".h5")
  print("Saved model to disk")

In [18]:
# Load model saved with the previous function
def load_model(filename):
  json_file = open(filename+'.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model
  loaded_model.load_weights(filename+".h5")
  print("Loaded model from disk")
  return loaded_model

In [19]:
'''
This function generates files each containing the data for a batch. The batches contain recordings of length snippet_len
and each sample is moved by 125 datapoints (0.25 seconds). Therefore, the samples have a lot of overlap. The idea was to do the preprocessing
beforehand and then load the data from storage (because they couldn't fit in the RAM). In the end, it turned out that loading the I/O operations
take longer than the preprocessing. I didn't use that since I sticked to samples of snippet_len=125
'''
def create_batches(data_path, save_path, scaler, batch_size=32, snippet_len=125, clean=True, channels=4):

  filenames = os.listdir(data_path)

  # initialization
  batch = np.zeros((batch_size, snippet_len, channels))
  batch_index = 0
  batch_num = 0

  for name in tqdm(filenames):
    ##### Accessing data from files and preprocession ####

    file_name = data_path + '/' + name
    file_signal = h5py.File(file_name, 'r')
    signal = file_signal['data'][()]
    file_signal.close()

    file_signal = self.scaler.transform(file_signal.T).T

    quality_name = 'Data/fetal_quality_assessment/quality_files/' + name[:-12] + '.quality'
    file_signal = h5py.File(quality_name, 'r')
    signal_quality = file_signal['data'][()]
    file_signal.close()
    signal_quality = remove_nan(signal_quality)
    signal_quality = signal_quality.flatten()

    # make it an integer multiple of snippet_len
    leftovers = signal.shape[1]%snippet_len
    if leftovers != 0:
      signal = signal[:,:-leftovers]
      signal_quality = signal_quality[snippet_len//125-1:-leftovers//125]
    else:
      signal_quality = signal_quality[snippet_len//125-1:]

    #### Creating batch files ######

    signal_index = 0

    while signal_index*125+snippet_len < signal.shape[1]:
    ####### Write batch file to disk #######

      if batch_index == batch_size:
        print('Batch ' + str(batch_num) + ' saved')

        batch = scale_data(batch, scaler, snippet_len)

        hf = h5py.File(save_path + '/' + 'Batch_{}'.format(batch_num)  + '.h5', 'w')
        hf.create_dataset('data', data=batch)
        hf.close()

        batch_index = 0
        batch_num += 1
        batch = np.zeros((batch_size, snippet_len, channels))


      if clean and signal_quality[signal_index] == 1:
        batch[batch_index] = signal[:,signal_index*125 : signal_index*125+snippet_len].T
        batch_index += 1
      elif not clean:
        batch[batch_index] = signal[:,signal_index*125 : signal_index*125+snippet_len].T
        batch_index += 1
        
      signal_index += 1


In [20]:
'''
NOT USED IN THE END
It loads each recording on memory and then feeds the neural network with batches until the recording is over. 
Then it loads the next one.
'''
class Custom_Generator_preprocessed(tf.keras.utils.Sequence) :

  def __init__(self, path, batch_size, shuffle=True) :

    self.filenames = [path + '/' + name for name in os.listdir(path)]
    self.shuffle = shuffle

    self.current_file_index = 0
    self.current_data = None
    self.load_data(self.current_file_index)

    self.slices = self.current_data.shape[0]//batch_size
    self.in_file_indices = np.arange(self.slices)
    self.step = self.current_data.shape[0]//self.slices

    self.on_epoch_end()
    self.load_data(self.current_file_index)




  def __len__(self) :
    return self.slices*len(self.filenames)
  

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.current_file_index = 0

    if self.shuffle is True:
        np.random.shuffle(self.filenames)
        np.random.shuffle(self.in_file_indices)


  # preload data, I can do in class variables
  def load_data(self, index):

    name = self.filenames[index]

    try:

      file_signal = h5py.File(name, 'r')
      data = file_signal['data'][()]
      file_signal.close()

      self.current_data = data
      print('File {} loaded..'.format(name))

    except KeyError as e:

      print('File {} was broken. Next file fed'.format(name))
      self.load_data(index+1, target)


  def __getitem__(self, idx) :

    index_in_file = self.in_file_indices[idx%self.slices]

    data = self.current_data[index_in_file*self.step : (index_in_file+1)*self.step]

    if idx%self.slices == self.slices - 1:
      self.current_file_index += 1
      self.load_data(self.current_file_index)

    return data, data

In [21]:
# generator = Custom_Generator_preprocessed('Data/fetal_quality_assessment/1250_clean_dataset', batch_size=256, shuffle=True)

In [22]:
'''
It creates batch files with only the necessary data to reconstruct them. 
Meaning, it doesn't contain the overlapping data, it only filters the bad data
It returns the recording with size (-1, 4) that will be transformed to the batch (batchsize, snippet_len, 4) after removing 
the low quality samples
'''
def preprocess_data(data_path, quality_path, save_path, scaler, batch_size, snippet_len):

  filenames = os.listdir(data_path)

  # recording length that we need to construct the signal if we do not count the low quality points
  min_recording_len = batch_size*125+snippet_len
  max_recording_len = 3*min_recording_len # arbitrary
  batch_num = 0 # just for the naming of the files

  for name in tqdm(filenames):
    ##### Accessing data from files and preprocession ####

    file_name = data_path + '/' + name
    file_signal = h5py.File(file_name, 'r')
    signal = file_signal['data'][()]
    file_signal.close()

    signal = scaler.transform(signal.T).T

    quality_name = 'Data/fetal_quality_assessment/quality_files/' + name[:-12] + '.quality'
    file_signal = h5py.File(quality_name, 'r')
    signal_quality = file_signal['data'][()]
    file_signal.close()
    signal_quality = remove_nan(signal_quality)
    signal_quality = signal_quality.flatten()

    #### Creating batch files ######

    # initialize batch
    batch = np.zeros((max_recording_len, 4))
    batch[0:snippet_len, :] = signal[:,0 : snippet_len].T

    signal_index = snippet_len//125 - 1   # the index in the quality file, it starts after the first snippet_len
    rejected_samples_indexes = []   # the indexes of the samples that will need to be rejected because they end in a low quality snippet
    batch_index = 0   # in which index in the current sample we are
    valid_samples = 0 # how many valid (with good quality ending) the current batch contains

    if signal_quality[signal_index] == 0:
      rejected_samples_indexes.append(batch_index)
    else:
      valid_samples += 1
    
    batch_index += 1

    while (signal_index+1)*125+snippet_len < signal.shape[1]:

    ####### Write batch file to disk #######
      if valid_samples == batch_size:
        # trim batch_zeros
        batch = batch[:batch_index*125+snippet_len, : ]

        hf = h5py.File(save_path + '/' + 'Batch_{}'.format(batch_num)  + '.h5', 'w')
        hf.create_dataset('data', data=batch)
        hf.create_dataset('rejected', data=np.array(rejected_samples_indexes))
        hf.close()

        print('Batch ' + str(batch_num) + ' saved')
        batch_num += 1

        # reinitialize
        batch = np.zeros((max_recording_len, 4))
        batch[0:snippet_len, :] = signal[:,signal_index*125 : signal_index*125+snippet_len].T

        rejected_samples_indexes = []   # the indexes of the samples that will need to be rejected because they end in a low quality snippet
        batch_index = 0   # in which index in the current sample we are
        valid_samples = 0 # how many valid (with good quality ending) the current batch contains

        if signal_quality[signal_index] == 1:
          rejected_samples_indexes.append(batch_index)
        else:
          valid_samples += 1
        
        batch_index += 1
    ##################################################

      try:
        # write 125 snippet to batch
        batch[batch_index*125+snippet_len : (batch_index+1)*125+snippet_len, :] = \
                                              signal[:, signal_index*125+snippet_len : (signal_index+1)*125+snippet_len].T

      except ValueError:
        batch = np.append(batch, np.zeros((min_recording_len, 4)), axis=0)

        batch[batch_index*125+snippet_len : (batch_index+1)*125+snippet_len, :] = \
                                              signal[:, signal_index*125+snippet_len : (signal_index+1)*125+snippet_len].T
    
      if signal_quality[signal_index] == 0:
        rejected_samples_indexes.append(batch_index)
      else:
        valid_samples += 1

      batch_index += 1
      signal_index += 1


In [23]:
'''
This creates a single file with three datasets:
Data: a numpy array with shape (-1, 125, 4)
bad_quality: a 1D numpy array containing the indices of bad data points
recording_endings: 1D numpy array containing the indicies of the endings of recordings
'''
def create_single_file_dataset(data_path, quality_path, save_path, scaler=None):

  filenames = os.listdir(data_path)

  # find total length of data in 125 segments

  total = 0
  for name in tqdm(filenames):

    if name[0] == '1':

      quality_name = quality_path + '/' + name[:-12] + '.quality'
      file_signal = h5py.File(quality_name, 'r')
      signal_quality = file_signal['data'][()]
      file_signal.close()
      signal_quality = signal_quality.flatten()

      total += len(signal_quality)


  data = np.zeros((total, 125, 4), dtype=np.float16)
  bad_quality = np.zeros(0, dtype=np.int)
  recording_endings = []
  index = 0


  for name in tqdm(filenames):
    ##### Accessing data from files and preprocession ####

    file_name = data_path + '/' + name
    file_signal = h5py.File(file_name, 'r')
    signal = file_signal['data'][()]
    file_signal.close()

    # shape = (-1,4)
    if scaler is not None:
      signal = scaler.transform(signal.T)
    else:
      signal = signal.T

    quality_name = quality_path + '/' + name[:-12] + '.quality'
    file_signal = h5py.File(quality_name, 'r')
    signal_quality = file_signal['data'][()]
    file_signal.close()
    signal_quality = remove_nan(signal_quality)
    signal_quality = signal_quality.flatten()

    # make it an integer multiple of 125
    leftovers = signal.shape[0]%125
    if leftovers != 0:
      signal = signal[:-leftovers, :]
      signal_quality = signal_quality[:-leftovers//125]

    signal = np.reshape(signal, newshape=(-1, 125, 4)) 

    bad_quality_temp = np.where(signal_quality == 0)[0] + index

    data[index : index + signal.shape[0]] = signal

    index += signal.shape[0]

    bad_quality = np.append(bad_quality, bad_quality_temp)
    recording_endings.append(index)

  data = data[:index]
  hf = h5py.File(save_path, 'w')
  hf.create_dataset('data', data=data)
  hf.create_dataset('bad_quality', data=bad_quality)
  hf.create_dataset('recording_endings', data=np.array(recording_endings))
  hf.close()


In [25]:
'''
This creates a single file with three datasets:
Data: a numpy array with shape (-1, 125, 4)
bad_quality: a 1D numpy array containing the indices of bad data points
recording_endings: 1D numpy array containing the indicies of the endings of recordings
'''
def create_single_file_dataset_non_fetal(data_path, quality_path, save_path, scaler):

  filenames = os.listdir(data_path)

  # find total length of data in 125 segments

  total = 0
  for name in tqdm(filenames):

    quality_name = quality_path + '/' + name[:-12] + '.quality'
    file_signal = h5py.File(quality_name, 'r')
    signal_quality = file_signal['data'][()]
    file_signal.close()
    signal_quality = signal_quality.flatten()

    total += len(signal_quality)


    data = np.zeros((total, 125, 4), dtype=np.float16)
    recording_endings = []
    index = 0


  for name in tqdm(filenames):
    ##### Accessing data from files and preprocession ####

    file_name = data_path + '/' + name
    file_signal = h5py.File(file_name, 'r')
    signal = file_signal['data'][()]
    file_signal.close()

    signal = c_trim_zeros_NEMO(signal)

    # shape = (-1,4)
    signal = scaler.transform(signal.T)

    # make it an integer multiple of 125
    leftovers = signal.shape[0]%125
    if leftovers != 0:
      signal = signal[:-leftovers, :]

    signal = np.reshape(signal, newshape=(-1, 125, 4)) 

    data[index : index + signal.shape[0]] = signal

    index += signal.shape[0]

    recording_endings.append(index)

  data = data[:index]
  hf = h5py.File(save_path, 'w')
  hf.create_dataset('data', data=data)
  hf.create_dataset('recording_endings', data=np.array(recording_endings))
  hf.close()


In [26]:
# create_single_file_dataset_non_fetal('Data/non_fetal_signals/malePatients', 'Data/quality_files', 
#                                      'Data/non_fetal.h5', load(open('Models/ALL_scaler.pkl', 'rb')))

In [27]:
# create_single_file_dataset('Data/train', 'Data/quality_files', 'Data/train_folder.h5', load(open('Models/ALL_scaler.pkl', 'rb')))

In [28]:
'''
Reconstructs the recordings from the dataset file.
INPUT:
dataset, contains the data in shape (-1, 125,4) - all recordings appended
bad_quality: it contains the indices of the low quality recordings
recording_endings: it contains the indices of the ends of each recording
OUTPUT
recordings_labels: list of tuples. Each tuple contains the recording in shape (-1,125,4)
and the labels with low quality -> 1 and high quality -> 0
'''
def reconstruct_recordings(dataset, bad_quality, recording_endings):
  recordings_labels = []
  for i in range(len(recording_endings)):
    start = 0 if i==0 else recording_endings[i-1]
    end = recording_endings[i]

    recording = dataset[start:end,:,:]

    bad_indices = bad_quality[np.where(bad_quality >= start)[0]]
    bad_indices = bad_indices[np.where(bad_indices < end)[0]]
    bad_indices = bad_indices - start

    quality = np.zeros(end-start)
    quality[bad_indices] = 1

    recordings_labels.append((recording, quality))
  
  return recordings_labels

In [29]:
'''
Reconstructs the recordings from the dataset file, it doesn't need bad_quality indices because all are bad
'''
def reconstruct_recordings_non_fetal(dataset, recording_endings):
  recordings_labels = []
  for i in range(len(recording_endings)):
    start = 0 if i==0 else recording_endings[i-1]
    end = recording_endings[i]

    recording = dataset[start:end,:,:]
    labels = np.ones(recording.shape[0])

    recordings_labels.append((recording, labels))

  return recordings_labels

In [30]:
'''
INPUT
recordings_labels: list of tuples. Each tuple contains the recording in shape (-1,125,4)
and the labels with low quality -> 1 and high quality -> 0
autoencoder: the keras model autoencoder that will be used to reconstruct the recordings
OUTPUT
errors_labels: List of tuples with each tuple containing the reconstruction errors and the associated label (low quality ->1)
'''
def get_errors_per_recording(recordings_labels, autoencoder):

  errors_labels = []

  for i in range(len(recordings_labels)):
    error = find_errors(recordings_labels[i][0], autoencoder)
    errors_labels.append((error, recordings_labels[i][1]))

  return errors_labels

In [31]:
'''
errors: the array containing all the errors of each 125 snippet. It should be flattened 1d
lookback: how many
return: output[i] = [error[t], error[t-1], ..., error[t-lookback-1] ]
'''
def create_features(errors, lookback):

  output = np.zeros(shape=(len(errors), lookback))
  for i in range(len(errors)):
    output[i, :np.min([lookback, i+1])] = np.array([errors[i-j] for j in range(np.min([lookback, i+1]))])

  return output

In [32]:
'''
errors: the array containing all the errors of each 125 snippet. It should be flattened 1d
lookback: how many
return: output[i] = [error[t], error[t-1], ..., error[t-lookback-1] ]
'''
def create_features_2d(errors, lookback):

  output = np.zeros(shape=(len(errors), lookback, 2))
  for i in range(len(errors)):
    output[i, :np.min([lookback, i+1]), :] = np.array([errors[i-j,:] for j in range(np.min([lookback, i+1]))])

  return output

In [33]:
'''
INPUT
errors_labels: List of tuples with each tuple containing the reconstruction errors and the associated label (low quality ->1)
lookback: how much historical data we will use
OUTPUT
data: processed data containing errors plus the lookback (overlapping)
labels: associated labels
'''
def create_dataset_from_errors(error_labels, lookback):

  data = np.zeros((0,lookback))
  labels = np.empty(0)
  for i in range(len(error_labels)):
    errors = error_labels[i][0]
    l = error_labels[i][1]
    features = create_features(errors, lookback)

    data = np.concatenate((data,features), axis=0)
    labels = np.concatenate((labels,l))
  
  return data, labels


In [34]:
# same as before but now it also includes errors of the reconstruction on the frequency domain
def create_dataset_from_errors_2d(error_labels, lookback):

  data = np.zeros((0,lookback,2))
  labels = np.empty(0)
  for i in tqdm(range(len(error_labels))):
    errors = error_labels[i][0]
    l = error_labels[i][1]
    features = create_features_2d(errors, lookback)

    data = np.concatenate((data,features), axis=0)
    labels = np.concatenate((labels,l))
  
  return data, labels

In [35]:
# it subsamples the dataset so that it is balanced
# ratio = majority/minority
def balance_subsample(data, labels, ratio=1):

  one_indices = np.where(labels == 1)[0]
  zero_indices = np.where(labels == 0)[0]

  indices = {0: zero_indices, 1: one_indices}

  minority = 0 if len(zero_indices) < len(one_indices) else 1
  majority = int(not minority)

  minority_data = data[indices[minority]]
  majority_data = data[indices[majority]]

  # subsample
  majority_data = majority_data[:int(ratio * minority_data.shape[0])]

  minority_labels = np.zeros(minority_data.shape[0])
  minority_labels[:] = minority

  majority_labels = np.zeros(majority_data.shape[0])
  majority_labels[:] = majority

  new_data = np.concatenate((minority_data, majority_data), axis=0)
  new_labels = np.concatenate((minority_labels, majority_labels), axis=0).flatten()

  new_data, new_labels = shuffle(new_data, new_labels)
  
  return new_data, new_labels


In [36]:
'''
It performs the fast fourier transform on our data and it returns the scaled magnitude of the fourier transform.
It also balances time and space complexity, it makes the algorithm slower but it consumes less RAM (because we didn't have enough RAM 
to transform all the data at once)
'''
def fourier_lowmem(data, slices=15):

  fourier = np.zeros((0, data.shape[1]//2,  data.shape[2]))
  for i in tqdm(range(0, data.shape[0], data.shape[0]//slices)):
    start = i
    end = i + data.shape[0]//slices

    f = np.abs(np.fft.fft(data[start:end], axis=1))[:,:data.shape[1]//2,:] * 1/data.shape[1]

    fourier = np.concatenate((fourier, f), axis=0)
    # clean_scipy_cache()

  return fourier

In [37]:
'''
It performs the real fast fourier transform It  balances time and space complexity, 
it makes the algorithm slower but it consumes less RAM (because we didn't have enough RAM 
to transform all the data at once)
'''
def real_fourier_lowmem(data, slices=20):

  fourier = np.zeros((data.shape[0], data.shape[1]//2 + 1, data.shape[2]))
  for i in tqdm(range(0, data.shape[0], data.shape[0]//slices)):
    start = i
    end = i + data.shape[0]//slices

    f = np.fft.rfft(data[start:end], axis=1)
    # clean_scipy_cache()

    fourier[start:end] = f

  return fourier

In [38]:
'''
It performs the inverse real fast fourier transform It  balances time and space complexity, 
it makes the algorithm slower but it consumes less RAM (because we didn't have enough RAM 
to transform all the data at once)
'''
def inv_real_fourier_lowmem(data, length, slices=20):

  signal = np.zeros((data.shape[0], length, data.shape[2]))
  for i in tqdm(range(0, data.shape[0], data.shape[0]//slices)):
    start = i
    end = i + data.shape[0]//slices

    f = np.fft.irfft(data[start:end], n=length, axis=1)

    signal[start:end] = f

  return signal

In [39]:
'''
INPUT
recordings_labels: list of tuples. Each tuple contains the recording in shape (-1,125,4)
and the labels with low quality -> 1 and high quality -> 0
OUTPUT
list of tuples. Each tuple contains the scaled fourier transform recording in shape (-1,125,4)
and the labels with low quality -> 1 and high quality -> 0
'''
def get_fourier(recordings_labels): 

  fourier_labels = []

  for i in range(len(recordings_labels)):
    r = recordings_labels[i][0]
    l = recordings_labels[i][1]
    f = np.abs(np.fft.fft(r, axis=1))[:,:r.shape[1]//2,:] * 1/r.shape[1]
    fourier_labels.append((f,l))
  
  return fourier_labels


In [40]:
'''
Create dataset that has the errors from both time and frequency domain
'''
def create_2d_dataset(recordings_labels, model_t, model_f, scaler_t=None, scaler_f=None):

  errors_labels = []

  for i in tqdm(range(len(recordings_labels))):
    rec = recordings_labels[i][0]
    lab = recordings_labels[i][1]
    four = np.abs(np.fft.fft(rec, axis=1))[:,:rec.shape[1]//2,:] #* 1/rec.shape[1]

    error_t = find_errors(rec, model_t)

    error_f = find_errors(four, model_f)

    if scaler_t != None and scaler_f != None:
      error_t = scaler_t.transform(np.expand_dims(error_t, axis=1)).flatten()
      error_f = scaler_f.transform(np.expand_dims(error_f, axis=1)).flatten()

    error = np.vstack((error_t, error_f)).T

    errors_labels.append((error, lab))

  return errors_labels

In [41]:
%%cython
cimport numpy as cnp
import numpy as np

# Cython implementation of generating the batch
cpdef cnp.ndarray c_get_batch(int index, cnp.ndarray data, int batch_len, int batch_size, \
                            int snippet_len, int window_step, cnp.ndarray bad_quality, cnp.ndarray recording_endings):

    cdef int batch_index = 0
    cdef int data_index = index*batch_len # beginning index of this batch
    cdef cnp.ndarray batch = np.zeros((batch_size, snippet_len, 4))
    cdef cnp.ndarray data_range

    while batch_index < batch_size:
      
      data_range = np.arange(data_index, data_index + snippet_len//window_step)

      if data_index + snippet_len//window_step not in bad_quality \
      and np.intersect1d(data_range, recording_endings).size == 0:

        batch[batch_index] = np.reshape(data[data_index: data_index + snippet_len//window_step], (-1, 4))

        batch_index += 1
    
      data_index += 1

    return batch



In [42]:
'''
errors: the array containing all the errors of each 125 snippet. It should be flattened 1d
lookback: how many
return: output[i] = [error[t], error[t-1], ..., error[t-lookback-1] ]
'''
def create_lookback_2d(errors, lookback):

  output = np.zeros(shape=(len(errors), lookback, 2))
  for i in range(len(errors)):
    output[i, :np.min([lookback, i+1]), :] = np.array([errors[i-j,:] for j in range(np.min([lookback, i+1]))])

  return output

In [43]:
# NOT USED IN THE END
# It loads all the data on the memory and then feeds the neural network with samples. Trying to find a balance between I/O operations and 
# preprocessing
class Custom_Generator_one_file(tf.keras.utils.Sequence) :

  def __init__(self, path, batch_size, snippet_len, window_step=125, shuffle=True) :

    file_signal = h5py.File(path, 'r')
    self.data = file_signal['data'][()]
    self.bad_quality = file_signal['bad_quality'][()]
    self.recording_endings = file_signal['recording_endings'][()]
    file_signal.close()

    self.shuffle = shuffle
    self.window_step = window_step

    self.batch_size = batch_size
    self.snippet_len = snippet_len

    # number of datapoints in a batch
    self.batch_len = self.snippet_len//self.window_step + self.batch_size - 1
    self.length = self.find_length()

    self.indices = np.arange(self.length)
    self.on_epoch_end()
    
  def __len__(self) :
    return self.length

  
  def find_length(self):
    length = (self.data.shape[0] - len(self.bad_quality) - len(self.recording_endings)*self.snippet_len//self.window_step)//self.batch_len
    return length
  

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    if self.shuffle == True:
        np.random.shuffle(self.indices)

  @numba.jit(forceobj=True, fastmath=True)
  def get_batch(self, index):

    batch_index = 0
    data_index = index*self.batch_len # beginning index of this batch
    batch = np.zeros((self.batch_size, self.snippet_len, 4))

    while batch_index < self.batch_size:
      
      data_range = np.arange(data_index, data_index + self.snippet_len//self.window_step)

      if data_index + self.snippet_len//self.window_step not in self.bad_quality \
      and np.intersect1d(data_range, self.recording_endings).size == 0:
        batch[batch_index] = np.reshape(self.data[data_index: data_index + self.snippet_len//self.window_step], (-1, 4))

        batch_index += 1
    
      data_index += 1

    return batch


  def __getitem__(self, idx) :

    index = self.indices[idx]
    # batch = self.get_batch(index)
    batch = c_get_batch(index, self.data, self.batch_len, self.batch_size, self.snippet_len, 
                        self.window_step, self.bad_quality, self.recording_endings)

    return batch, batch

In [44]:
'''
function that returns consecutive high or low quality data
length: segment length in # of data points
wanted_quality:  (1 low, 0 high)
recording: the recording with shape (-1,125,4)
labels: the corresponding labels (1 low, 0 high)
'''
def consecutive_data(length, recording, labels, wanted_quality=0):

  segments = []
  signal_index = 0

  segments_in_length = length//recording.shape[1]

  while signal_index < len(labels):

    current_segment = np.zeros((segments_in_length, recording.shape[1], recording.shape[2]))
    current_segment_index = 0

    while current_segment_index < segments_in_length:

      if signal_index + current_segment_index < len(labels):

        if wanted_quality == labels[signal_index + current_segment_index]:
          current_segment[current_segment_index] = recording[signal_index + current_segment_index]
        
        else:
          signal_index += 1
          current_segment_index += 1
          break
        
      else:
        signal_index += 1
        current_segment_index += 1
        break

      if current_segment_index+1 == segments_in_length:
        segments.append(current_segment.reshape(1, length, recording.shape[2]))

      signal_index += 1
      current_segment_index += 1


  segments_array = np.concatenate(segments, axis=0)
  
  return segments_array


In [45]:
'''
This function generates a dataset consisting of consecutive high quality segments of a specific length
INPUT
dataset: contains the data in shape (-1, 125,4) - all recordings appended
bad_quality: it contains the indices of the low quality recordings
recording_endings: it contains the indices of the ends of each recording
length: The length of the segments of consecutive high quality data, it must be a multiple of 125
how_many_files: Limits the number of files we will use, it accounts for memory limitations
OUTPUT
The dataset in shape (-1, length, channels)
'''
def consecutive_clean_dataset(dataset, bad_quality, recording_endings, length, how_many_files=60):

  reconstructed = reconstruct_recordings(dataset, bad_quality, recording_endings)[:how_many_files]

  # total_bad_quality = bad_quality.size
  # total_good_quality = dataset.shape[0] - total_bad_quality
  # total_size = int(total_good_quality * length/dataset.shape[1])
  # segments = np.zeros((total_size, length, dataset.shape[2]))
  # current_index = 0

  segments = np.zeros((0, length, dataset.shape[2]))

  for i in tqdm(range(len(reconstructed))):

    recording = reconstructed[i][0]
    labels = reconstructed[i][1]

    s = consecutive_data(length, recording, labels)

    segments = np.concatenate((segments, s), axis=0)

    # segments[current_index : current_index+s.shape[0]] = s
    # current_index += s.shape[0]

  
  return segments

In [46]:
# # Count how much data we have
# filenames = os.listdir('Data/Clean/quality_files')

# total = 0
# for name in tqdm(filenames):

#   if name[0] == '1':

#     quality_path = 'Data/Clean/quality_files/' + name
#     file_signal = h5py.File(quality_path, 'r')
#     signal_quality = file_signal['data'][()]
#     file_signal.close()
#     signal_quality = signal_quality.flatten()

#     total += len(signal_quality)

# print('We have {} quality points, meaning {} datapoints'.format(str(total), str(total*125)))


In [47]:
# WE USED 72 FILES to create the scaler
# # FIT SCALER 

# filenames = os.listdir(r'Data/fetal_quality_assessment/all_fetal_signals')
# np.random.shuffle(filenames)
# all_signals = np.zeros((4,0))
# used_files = []

# scaler = RobustScaler()
# for name in tqdm(filenames):

#   file_signal = h5py.File(r'Data/fetal_quality_assessment/all_fetal_signals/{}'.format(name), 'r')
#   signal = file_signal['data'][()]
#   file_signal.close()

#   all_signals = np.concatenate((all_signals, signal), axis=1)
#   scaler.fit(all_signals.T)
#   dump(scaler, open('ALL_scaler.pkl', 'wb'))

#   used_files.append(name)
#   print('We used %d files' % len(used_files))